In [1]:
import os
import pandas as pd
BATCHES_FOLDER = "MTurkBatchesResults"
OUTPUT_FILEMANE = "output.csv"

This code finds the "truth" - annotators agreements and stores to file

In [2]:
mturk_batches = ["{}/{}".format(BATCHES_FOLDER, fl) for fl in os.listdir(BATCHES_FOLDER) if "truth.csv" in fl]#fl.endswith(".csv")]
print(mturk_batches)
#modified by hand
truth_df = pd.read_csv("{}/{}".format(BATCHES_FOLDER, "truth.csv"))
# mturk_batches += ["{}/{}".format(BATCHES_FOLDER, "truth.csv") for i in range(5)]

['MTurkBatchesResults/truth.csv']


In [3]:
truth_df.head(10)

,company,url,climate_confidence,company_confidence,sentiment,content,extract,title,comments
0,Goodyear Tire & Rubber Co,http://www.performancetopeka.com/shop-for-tire...,0.0,2.0,NaN,"<HTML lang=""en-US"">\n<style type=""text/css"">\n...",The <b>Goodyear Tire &amp; Rubber Company</b> ...,"Shop Goodyear Tires in Topeka, KS :: Performan...",d
1,IBM,https://hbr.org/2004/12/leading-change-when-bu...,0.0,0.0,NaN,"<HTML class="""">\n<style type=""text/css"">\nA:be...",By the time Sam Palmisano took over as CEO in ...,Leading Change When Business Is Good,d
2,RosNeft,http://www.leonardocompany.com/en/-/accordo-ru...,0.0,1.0,NaN,"<HTML class=""ltr"" dir=""ltr"" lang=""en-GB"">\n<st...","... Leonardo-Finmeccanica, Russian Helicopters...",New agreement for Leonardo-Finmeccanica's heli...,d
3,Amazon.com Inc.,https://sloanreview.mit.edu/article/digital-tr...,0.0,1.0,Neutral,"<HTML class=""no-js mit-user "" lang=""en-US"" pre...",... Digital Transformation Sustainability <b>C...,Digital Transformation on Purpose,d
4,ABB,http://microgridmedia.com/abb-indian-institute...,0.0,2.0,NaN,"<HTML lang=""en-US"" prefix=""og: http://ogp.me/n...",... Both India and multinational power enginee...,"ABB, Indian Institute of Technology Microgrid ...",d
5,Toyota Motor Corp.,http://www.autonews.com/article/20180209/RETAI...,0.0,2.0,NaN,"<HTML lang=""en-US"">\n<style type=""text/css"">\n...",... That is the motto that accompanies <b>Toyo...,Toyota launches ad campaign tied to 2018 Winte...,sport
6,Daimler AG,http://www.greencarcongress.com/2018/03/201803...,0.0,1.0,NaN,"<HTML>\n<style type=""text/css"">\nA:before { co...",... The BMW Group and <b>Daimler AG</b> have s...,BMW Group and Daimler AG agree to combine mobi...,d
7,EDS Corp.,https://www.nyserda.ny.gov/climaid,0.0,0.0,NaN,"<HTML lang=""en"" xml:lang=""en"" xmlns=""http://ww...",Responding to <b>Climate Change</b> in New Yor...,Responding Climate Change in New York State (C...,-
8,Bunge,https://www.nature.org/ourinitiatives/regions/...,0.0,1.0,NaN,"<HTML lang=""en"">\n<style type=""text/css"">\nA:b...","In his post-banking business life, Peter was a...",Peter Wheeler Bio | The Nature Conservancy,done
9,Goodyear Tire & Rubber Co,http://www.east-sideautomotive.com/tire-brands...,0.0,2.0,NaN,"<HTML lang=""en-US"">\n<style type=""text/css"">\n...",The <b>Goodyear Tire &amp; Rubber Company</b> ...,"Shop Goodyear Tires Stoughton, WI Edgerton, WI...",d


In [4]:
import numpy as np
df = pd.concat([pd.read_csv(batch) for batch in mturk_batches])
# df = df[df['sentiment'].notnull()]

df = df.dropna(subset = ['company_confidence'])
truth_df = truth_df.dropna(subset = ['company_confidence'])
# df = df[np.isfinite(df['sentiment'])]
urls = df.groupby("url", as_index=False)

In [5]:
# pd.options.display.max_colwidth = 10000
# row = df.loc[df[''] == 'http://bellona.org/news/arctic/2017-04-trump-like-friend-of-putin-tells-him-climate-change-isnt-man-made']
# row['']
# df.columns
len(df)

444

In [6]:
columns = ['url', 'company', 'short_text', 'title', 'content', 'climate_confidence', 'company_confidence', 'sentiment']
result_df = pd.DataFrame(columns=columns)
# agree_level = []
cf = []

for num, url in enumerate(list(urls.groups.keys())):
    results = df.loc[df['url'] == url]
#     if 'w.ohio.com/akron/writers/katie-byard/goodyear-bridgestone-team-up-to-distribute-tires-nationallly-new-tirehub-will-be-one-of-the-largest-such-joint-ventures-in-the-country' in url:
#         import ipdb; ipdb.set_trace()
    key = results['extract'].keys()[0]
    vals = [url, results['company'][key], results['extract'][key], results['title'][key], results['content'][key]]
    for column_name in ['climate_confidence', 'company_confidence', 'sentiment']:
        #agree level == 2

        if any(truth_df['url'] == url) and truth_df.loc[truth_df['url'] == url][column_name].iloc[0] != np.nan:
            vals.append(truth_df.loc[truth_df['url'] == url][column_name].iloc[0])
#             if column_name =='company_confidence' and np.isnan(truth_df.loc[truth_df[''] == url][column_name].iloc[0]):
#                 import ipdb; ipdb.set_trace()
#                 cf.append(truth_df.loc[truth_df[''] == url][column_name].iloc[0])
            continue
        
        counter = results[column_name].value_counts()
        if len(counter) > 0:
            highest = max(counter.values)
            #         agree_level.append("{} of {}".format(highest, sum(counter.values)))
            res = [k for k, v in counter.items() if v == highest]
            vals.append(res[0])
            continue
        vals.append(None)
    result_df.loc[num] = vals
# result_df['agree_level'] = agree_level

In [7]:
len(result_df)

410

In [8]:
result_df['company'].unique()

# df.loc[result_df['Штзгею'] == result_df['company_confidence'].unique()[-1]]
# dr

array(['Apple Inc', 'RosNeft', 'Ford Motor Co.', 'GazProm', 'Petro China',
       'BASF AG', 'Royal Dutch Shell PLC', 'BMW',
       'Goodyear Tire & Rubber Co', 'Toyota Motor Corp.',
       'Exxon Mobil Corp.', 'Cargill Inc', 'Ineos', 'EDS Corp.',
       'General Motors Corp.', 'ABB', 'Nestle SA', 'Petrobras',
       'Duke Energy', 'Continental AG', 'Sumitomo Rubber Industries',
       'Dow Chemical', 'Amazon.com Inc.', 'Volkswagen AG', 'Daimler AG',
       'AuFeminin.com', 'PepsiCo Inc.', 'Bridgestone Corp.', 'IBM',
       'Bayer AG', 'Michelin', 'Johnson & Johnson', 'Chevron Corp.',
       'BP plc', 'Microsoft Corp.', 'Bunge'], dtype=object)

In [9]:
result_df.to_csv(OUTPUT_FILEMANE, index=False)